In [1]:
import sys
import os
import importlib
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'import'))
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'module'))

import import_default
import import_database
import import_other
import draw_ta_chart

importlib.reload(import_default)
importlib.reload(import_database)
importlib.reload(import_other)
importlib.reload(draw_ta_chart)

from import_default import *
from import_database import *
from import_other import *
from draw_ta_chart import *

In [2]:
# Dữ liệu tham chiếu
date_series = get_mongo_collection(ref_db, 'date_series')
name_map = get_mongo_collection(ref_db, "name_map")
name_map_dict = name_map.set_index('code')['full_name'].to_dict()
full_stock_classification_df = get_mongo_collection(ref_db, 'full_stock_classification')
current_quarter_classification = get_mongo_collection(ref_db, 'current_quarter_classification')

# Dữ liệu cổ phiếu
projection = {"_id": 0,"date": 1,"ticker": 1,"open": 1,"high": 1,"low": 1,"close": 1, "volume": 1, "cap": 1}
today_stock_df = get_mongo_collection(stock_db, "today_stock", projection=projection)
history_stock_df = get_mongo_collection(stock_db, "history_stock", projection=projection)
full_stock_df = pd.concat([today_stock_df, history_stock_df], axis=0, ignore_index=True)
full_stock_df['industry'] = full_stock_df['ticker'].map(full_stock_classification_df.set_index('ticker')['industry_name'])

# Dữ liệu chỉ số
projection = {"_id": 0,"date": 1,"ticker": 1,"open": 1,"high": 1,"low": 1,"close": 1, "volume": 1}
today_index_df = get_mongo_collection(stock_db, "today_index", projection=projection)
history_index_df = get_mongo_collection(stock_db, "history_index", projection=projection)
full_index_df = pd.concat([today_index_df, history_index_df], axis=0, ignore_index=True)

# Dữ liệu nhóm cổ phiếu
today_group_df = get_mongo_collection(stock_db, "today_group")
history_group_df = get_mongo_collection(stock_db, "history_group")
full_group_df = pd.concat([today_group_df, history_group_df], axis=0)

In [3]:
market_period_dict = {
    "period_1": {
        "name": "Sóng hồi phục từ đáy COVID-19",
        "start_date": "2020-04-01",
        "end_date": "2020-07-29",
        "trend": "uptrend",
        "phase": 1
    },
    "period_2": {
        "name": "Sóng tăng vượt đỉnh tiền COVID-19",
        "start_date": "2020-07-29",
        "end_date": "2021-01-29",
        "trend": "uptrend",
        "phase": 2
    },
    "period_3": {
        "name": "Sóng tăng thứ hai sau COVID-19",
        "start_date": "2021-01-29",
        "end_date": "2021-07-20",
        "trend": "uptrend",
        "phase": 3
    },
    "period_4": {
        "name": "Sóng tăng lên đỉnh lịch sử 1500 điểm",
        "start_date": "2021-07-20",
        "end_date": "2022-04-04",
        "trend": "uptrend",
        "phase": 4
    },
    "period_5": {
        "name": "Nhịp điều chỉnh từ đỉnh 1500 điểm",
        "start_date": "2022-04-04",
        "end_date": "2022-07-11",
        "trend": "downtrend",
        "phase": 1
    },
    "period_6": {
        "name": "Sóng hồi trong xu hướng giảm 2022",
        "start_date": "2022-07-11",
        "end_date": "2022-09-05",
        "trend": "downtrend",
        "phase": 2
    },
    "period_7": {
        "name": "Nhịp giảm tạo đáy xu hướng 2022",
        "start_date": "2022-09-05",
        "end_date": "2022-11-16",
        "trend": "downtrend",
        "phase": 3
    },
    "period_8": {
        "name": "Sóng tăng mạnh từ đáy 873 điểm",
        "start_date": "2022-11-16",
        "end_date": "2022-12-06",
        "trend": "uptrend",
        "phase": 1
    },
    "period_9": {
        "name": "Giai đoạn tích lũy sau phục hồi",
        "start_date": "2022-12-06",
        "end_date": "2023-04-26",
        "trend": "sideways",
        "phase": 1
    },
    "period_10": {
        "name": "Sóng tăng chính năm 2023",
        "start_date": "2023-04-26",
        "end_date": "2023-09-12",
        "trend": "uptrend",
        "phase": 1
    },
    "period_11": {
        "name": "Nhịp điều chỉnh mạnh cuối năm 2023",
        "start_date": "2023-09-12",
        "end_date": "2023-11-01",
        "trend": "downtrend",
        "phase": 1
    },
    "period_12": {
        "name": "Sóng tăng cuối 2023 - đầu 2024",
        "start_date": "2023-11-01",
        "end_date": "2024-03-08",
        "trend": "uptrend",
        "phase": 1
    },
    "period_13": {
        "name": "Giai đoạn Sideway kéo dài (2024)",
        "start_date": "2023-11-01",
        "end_date": "2025-01-13",
        "trend": "sideways",
        "phase": 1
    },
    "period_14": {
        "name": "Sóng tăng đầu 2025 (trước thông tin thuế)",
        "start_date": "2025-01-13",
        "end_date": "2025-03-25",
        "trend": "uptrend",
        "phase": 1
    },
    "period_15": {
        "name": "Giảm mạnh do thông tin thuế",
        "start_date": "2025-03-25",
        "end_date": "2025-04-09",
        "trend": "downtrend",
        "phase": 1
    },
    "period_16": {
        "name": "Sóng phục hồi sau thông tin thuế",
        "start_date": "2025-04-09",
        "end_date": "2025-06-10",
        "trend": "uptrend",
        "phase": 1
    },
    "period_17": {
        "name": "Sóng tăng vượt đỉnh",
        "start_date": "2025-06-10",
        "end_date": "2025-07-23",
        "trend": "uptrend",
        "phase": 2
    }
}

In [4]:
ref_industry_list = ['ban_le', 'bds', 'dv_hatang', 'van_tai']
ref_stocks_list = ['MWG', 'FPT', 'DGW', 'GAS', 'VRE']

pet_df = full_stock_df[full_stock_df['ticker'] == 'PET'].set_index('date').sort_index(ascending=False)
pet_df = pet_df[['close']].rename(columns={'close': 'PET'})

pet_price_df = pet_df.copy()
for stock in ref_stocks_list:
    pet_price_df[stock] = full_stock_df[full_stock_df['ticker'] == stock].set_index('date')['close']
for industry in ref_industry_list:
    pet_price_df[industry] = full_group_df[full_group_df['ticker'] == industry][['date', 'close']].set_index('date')
pet_price_df['VNINDEX'] = full_index_df[full_index_df['ticker'] == 'VNINDEX'][['date', 'close']].set_index('date')
pet_price_df = pet_price_df.reset_index()

pet_periods_df = pd.DataFrame.from_dict(market_period_dict, orient='index')
pet_periods_df.reset_index(inplace=True, names='period')
pet_periods_df['start_date'] = pd.to_datetime(pet_periods_df['start_date'])
pet_periods_df['end_date'] = pd.to_datetime(pet_periods_df['end_date'])

for ticker in pet_price_df.columns[1:]:
    temp_df = pet_periods_df.copy()
    temp_df['start_price'] = pet_periods_df.merge(pet_price_df, left_on='start_date', right_on='date', how='left')[ticker]
    temp_df['end_price'] = pet_periods_df.merge(pet_price_df, left_on='end_date', right_on='date', how='left')[ticker]
    temp_df['change'] = (temp_df['end_price'] - temp_df['start_price']) / temp_df['start_price']
    pet_periods_df[f'change_{ticker}'] = temp_df['change']
pet_periods_df.to_excel('../output/pet_periods.xlsx', index=False)


In [5]:
projection = {
    "_id": 0,
    "date": 1,
    "ticker": 1,
    "open": 1,
    "high": 1,
    "low": 1,
    "close": 1,
    "pct_change": 1,
    "diff": 1,
    "volume": 1,
    "option": 1,
    "SMA_20": 1,
    "SMA_60": 1,
    "RSI_14": 1,
    "week_open": 1,
    "month_prev_high": 1,
    "month_prev_low": 1,
    "month_open": 1,
    "quarter_prev_high": 1,
    "quarter_prev_low": 1,
    "quarter_open": 1,
    "year_open": 1,
    "MFIBO_0382": 1,
    "MFIBO_0500": 1,
    "MFIBO_0618": 1,
    "QFIBO_0382": 1,
    "QFIBO_0500": 1,
    "QFIBO_0618": 1,
	"YFIBO_0382": 1,
    "YFIBO_0500": 1,
    "YFIBO_0618": 1,
}



In [8]:
stock_name = 'VGS'

today_stock_df = get_mongo_collection(stock_db, "today_stock", find_query={"ticker": stock_name}, projection=projection)
history_stock_df = get_mongo_collection(stock_db, "history_stock", find_query={"ticker": stock_name}, projection=projection)
full_stock_ta_df = pd.concat([today_stock_df, history_stock_df], axis=0, ignore_index=True).iloc[:120]
image_name = f'TA_{stock_name}.png'

line_name_dict = {
    'SMA_20': 'SMA 20',
    'SMA_60': 'SMA 60',
    'week_open': 'WEEK OPEN',
    'month_open': 'MONTH OPEN',
    'quarter_open': 'QUARTER OPEN',
    'year_open': 'YEAR OPEN',
    'month_prev_high': 'LAST MHIGH',
    'month_prev_low': 'LAST MLOW',
    'quarter_prev_high': 'LAST QHIGH',
    'quarter_prev_low': 'LAST QLOW',
    'MFIBO_0382': 'MFIBO 0.382',
    'MFIBO_0500': 'MFIBO 0.500',
    'MFIBO_0618': 'MFIBO 0.618',
    'QFIBO_0382': 'QFIBO 0.382',
    'QFIBO_0500': 'QFIBO 0.500',
    'QFIBO_0618': 'QFIBO 0.618',
	'YFIBO_0382': 'YFIBO 0.382',
    'YFIBO_0500': 'YFIBO 0.500',
    'YFIBO_0618': 'YFIBO 0.618'
}

chart_config = create_chart_config(
	title_font_size=30,
	axis_font_size=24,
	tag_font_size=24,
	price_tag_font_size=24,
    min_spacing_ratio = 0.042,
	margin=dict(l=20, r=220, t=20, b=20, pad=10)
)

max_attempts = 5
for attempt in range(max_attempts):
    try:
        fig, fig_images = create_financial_chart(
            full_stock_ta_df.sort_values('date'), # DataFrame của bạn
            width=1400,
            height=1200,
            line_name_dict=line_name_dict,
            line_columns=list(line_name_dict.keys()),
            chart_config=chart_config,
            symbol_name=stock_name,
            path='../output',
            image_name = image_name
        )
        break
    except Exception as e:
        if attempt == max_attempts - 1:
            raise e
full_stock_ta_df.iloc[:60].to_csv(f'../output/{stock_name}.csv', index=False, sep='|', lineterminator='\n')
fig.show()